In [3]:
# Exercise 1

from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

# Koneksi ke MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

# Operasi bulk: insert beberapa dokumen
operations = [
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'})
]
courses_collection.bulk_write(operations)
print('Courses inserted successfully.')


Courses inserted successfully.


In [14]:
# Task 1

from pymongo import MongoClient, InsertOne

client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

# Hapus data lama agar tidak duplikat
courses_collection.delete_many({})

# Insert beberapa data courses dengan daftar mahasiswa terdaftar
dataset = [
    InsertOne({
        'course': 'Math 101',
        'department': 'Mathematics',
        'enrollments': 30,
        'students': ['Ave', 'Bobby', 'Sinta', 'Rocky']
    }),
    InsertOne({
        'course': 'CS 102',
        'department': 'Computer Science',
        'enrollments': 25,
        'students': ['Dewi', 'Tommy', 'Sarah', 'Nora']
    }),
    InsertOne({
        'course': 'History 201',
        'department': 'History',
        'enrollments': 20,
        'students': ['Lina', 'Yoga']
    }),
    InsertOne({
        'course': 'Physics 202',
        'department': 'Physics',
        'enrollments': 15,
        'students': ['Maria', 'Anton', 'Callysta']
    })
]

result = courses_collection.bulk_write(dataset)
print(f"{result.inserted_count} courses inserted successfully.\n")

# Menampilkan data yang sudah dimasukkan
print("===== LIST OF COURSES =====".center(70))
print(f"\n{'Course Name':<15} {'Department':<20} {'Enrollments':<12} {'Students'}")
print("-" * 70)

for doc in courses_collection.find():
    students = ", ".join(doc['students'])
    print(f"{doc['course']:<15} {doc['department']:<20} {doc['enrollments']:<12} {students}")

print("-" * 70)
print("All course data displayed successfully.")


4 courses inserted successfully.

                     ===== LIST OF COURSES =====                      

Course Name     Department           Enrollments  Students
----------------------------------------------------------------------
Math 101        Mathematics          30           Ave, Bobby, Sinta, Rocky
CS 102          Computer Science     25           Dewi, Tommy, Sarah, Nora
History 201     History              20           Lina, Yoga
Physics 202     Physics              15           Maria, Anton, Callysta
----------------------------------------------------------------------
All course data displayed successfully.


In [15]:
# Exercise 2

# Filter: kursus dengan enrollments > 20
for course in courses_collection.find({'enrollments': {'$gt': 20}}):
    print(course)

# Filter: departemen tertentu
for course in courses_collection.find({'department': {'$in': ['Computer Science', 'Mathematics']}}):
    print(course)


{'_id': ObjectId('690c13b77037cc533381432e'), 'course': 'Math 101', 'department': 'Mathematics', 'enrollments': 30, 'students': ['Ave', 'Bobby', 'Sinta', 'Rocky']}
{'_id': ObjectId('690c13b77037cc533381432f'), 'course': 'CS 102', 'department': 'Computer Science', 'enrollments': 25, 'students': ['Dewi', 'Tommy', 'Sarah', 'Nora']}
{'_id': ObjectId('690c13b77037cc533381432e'), 'course': 'Math 101', 'department': 'Mathematics', 'enrollments': 30, 'students': ['Ave', 'Bobby', 'Sinta', 'Rocky']}
{'_id': ObjectId('690c13b77037cc533381432f'), 'course': 'CS 102', 'department': 'Computer Science', 'enrollments': 25, 'students': ['Dewi', 'Tommy', 'Sarah', 'Nora']}


In [18]:
# Exercise 2 (Tampilan Tabel Rapi)

# Filter: kursus dengan enrollments > 20
print("\n" + "=== COURSES WITH ENROLLMENTS > 20 ===".center(70))
print(f"\n{'Course Name':<15} {'Department':<20} {'Enrollments':<12}")
print("-" * 50)

for course in courses_collection.find({'enrollments': {'$gt': 20}}):
    print(f"{course['course']:<15} {course['department']:<20} {course['enrollments']:<12}")

print("-" * 50)



                === COURSES WITH ENROLLMENTS > 20 ===                 

Course Name     Department           Enrollments 
--------------------------------------------------
Math 101        Mathematics          30          
CS 102          Computer Science     25          
--------------------------------------------------


In [19]:
# Task 2

# Filter: departemen tertentu
print("\n" + "=== COURSES IN SELECTED DEPARTMENTS ===".center(70))
print(f"\n{'Course Name':<15} {'Department':<20} {'Enrollments':<12}")
print("-" * 50)

for course in courses_collection.find({'department': {'$in': ['Computer Science']}}):
    print(f"{course['course']:<15} {course['department']:<20} {course['enrollments']:<12}")

print("-" * 50)
print("Query results displayed successfully.")



               === COURSES IN SELECTED DEPARTMENTS ===                

Course Name     Department           Enrollments 
--------------------------------------------------
CS 102          Computer Science     25          
--------------------------------------------------
Query results displayed successfully.


In [23]:
# Exercise 3

from pprint import pprint

print("\n" + "=== AVERAGE ENROLLMENT PER DEPARTMENT ===".center(80))

# Pipeline untuk menghitung rata-rata enrollments
pipeline = [
    {'$group': {'_id': '$department', 'average_enrollment': {'$avg': '$enrollments'}}}
]

# --- Versi JSON (hasil mentah) ---
print("\n--- JSON Format ---\n")
for result in courses_collection.aggregate(pipeline):
    pprint(result)

# --- Versi Tabel Rapi ---
print("\n--- Table Format ---\n")
print(f"{'Department':<25} {'Average Enrollment':<20}")
print("-" * 45)
for result in courses_collection.aggregate(pipeline):
    print(f"{result['_id']:<25} {round(result['average_enrollment'], 2):<20}")
print("-" * 45)



                   === AVERAGE ENROLLMENT PER DEPARTMENT ===                    

--- JSON Format ---

{'_id': 'History', 'average_enrollment': 20.0}
{'_id': 'Mathematics', 'average_enrollment': 30.0}
{'_id': 'Physics', 'average_enrollment': 15.0}
{'_id': 'Computer Science', 'average_enrollment': 25.0}

--- Table Format ---

Department                Average Enrollment  
---------------------------------------------
Mathematics               30.0                
History                   20.0                
Computer Science          25.0                
Physics                   15.0                
---------------------------------------------


In [24]:
# Task 3

print("\n" + "=== MAXIMUM ENROLLMENT PER DEPARTMENT ===".center(80))

# Pipeline untuk mencari nilai maksimum
pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}}
]

# Versi Tabel Rapi
print(f"\n{'Department':<25} {'Max Enrollment':<20}")
print("-" * 45)
for result in courses_collection.aggregate(pipeline):
    print(f"{result['_id']:<25} {result['max_enrollment']:<20}")
print("-" * 45)



                   === MAXIMUM ENROLLMENT PER DEPARTMENT ===                    

Department                Max Enrollment      
---------------------------------------------
Mathematics               30                  
History                   20                  
Computer Science          25                  
Physics                   15                  
---------------------------------------------


In [29]:
# Exercise 4

print("\n" + "=== PROJECT - RENAME AND SELECT FIELDS ===".center(80))

pipeline = [
    {'$project': {'_id': 0, 'course_name': '$course', 'department_name': '$department', 'enrollments': 1}}
]

# --- Versi JSON ---
print("\n--- JSON Format ---\n")
for result in courses_collection.aggregate(pipeline):
    pprint(result)

# --- Versi Tabel Rapi ---
print("\n--- Table Format ---\n")
print(f"{'Course Name':<20} {'Department Name':<25} {'Enrollments':<12}")
print("-" * 60)
for result in courses_collection.aggregate(pipeline):
    print(f"{result['course_name']:<20} {result['department_name']:<25} {result['enrollments']:<12}")
print("-" * 60)



                   === PROJECT - RENAME AND SELECT FIELDS ===                   

--- JSON Format ---

{'course_name': 'Math 101', 'department_name': 'Mathematics', 'enrollments': 30}
{'course_name': 'CS 102',
 'department_name': 'Computer Science',
 'enrollments': 25}
{'course_name': 'History 201', 'department_name': 'History', 'enrollments': 20}
{'course_name': 'Physics 202', 'department_name': 'Physics', 'enrollments': 15}

--- Table Format ---

Course Name          Department Name           Enrollments 
------------------------------------------------------------
Math 101             Mathematics               30          
CS 102               Computer Science          25          
History 201          History                   20          
Physics 202          Physics                   15          
------------------------------------------------------------


In [31]:
# Task 4

print("\n" + "=== ADDFIELDS - ENROLLMENT CATEGORY ===".center(80))

pipeline = [
    {
        '$addFields': {
            'enrollment_category': {
                '$cond': {
                    'if': {'$gt': ['$enrollments', 20]},
                    'then': 'High',
                    'else': 'Low'
                }
            }
        }
    },
    {'$project': {'_id': 0, 'course': 1, 'department': 1, 'enrollments': 1, 'enrollment_category': 1}}
]

# Versi Tabel Rapi
print(f"\n{'Course':<20} {'Department':<25} {'Enrollments':<12} {'Category':<10}")
print("-" * 70)
for result in courses_collection.aggregate(pipeline):
    print(f"{result['course']:<20} {result['department']:<25} {result['enrollments']:<12} {result['enrollment_category']:<10}")
print("-" * 70)
print("Aggregation results displayed successfully.".center(80))



                    === ADDFIELDS - ENROLLMENT CATEGORY ===                     

Course               Department                Enrollments  Category  
----------------------------------------------------------------------
Math 101             Mathematics               30           High      
CS 102               Computer Science          25           High      
History 201          History                   20           Low       
Physics 202          Physics                   15           Low       
----------------------------------------------------------------------
                  Aggregation results displayed successfully.                   
